## ■ 7장. 데이터 프레임의 다양한 응용 복습

1. 데이터 프레임에 함수를 매핑
    - apply
    - applymap
    - pipe
    
    
2. 열을 재구성
    - dataframe객체[재구성할 열 이름 리스트]


3. 열분리
    - dataframe.str.split('-')


4. 필터링
    - dataframe객체[mask]  # mask <-- True or False 에 해당하는 Mask
    - isin() 메소드


5. 데이터 프레임 합치기
    - dataframe.concat
    - dataframe.merge
    - dataframe.join
    

6. 그룹연산
    - 1단계 : 분할(split)  
    질문 : 운임클래스 3개중에서 가장 생존율이 높은 클래스는 어디인가 ?
    
    - 2단계 : 집계 및 변환, 필터링하는데 필요한 메소드를 적용
    - 3단계 : 2단계의 처리결과를 하나로 결합



## ■ 그룹연산
   - df.groupby
   - 그룹객체.함수()
   - 그룹객체.agg

## ■ 그룹객체.agg

문법 : 그룹객체.agg(매핑함수)

In [1]:
# 예제1.

"""
SQL>
SELECT deptno, sum(sal)
    from emp
    group by deptno;
"""

# Pandas

import pandas as pd

emp = pd.read_csv("d:\\data\\emp.csv")
df = emp.loc[:, ['sal', 'deptno']]
print(df, '\n')

grouped = df.groupby('deptno')
print(grouped.agg(sum)) # 그룹객체 : grouped // 매핑함수 : sum
print(grouped.sum())

     sal  deptno
0   5000      10
1   2850      30
2   2450      10
3   2975      20
4   1250      30
5   1600      30
6   1500      30
7    950      30
8   1250      30
9   3000      20
10   800      20
11  3000      20
12  1100      20
13  1300      10 

          sal
deptno       
10       8750
20      10875
30       9400
          sal
deptno       
10       8750
20      10875
30       9400


## 문제182. 아래의 SQL을 판다스로 구현하시오!

In [6]:
"""
SQL>
select deptno, job, sum(sal)
    from emp
    grouped by deptno,job;
"""

# pandas

import pandas as pd

emp = pd.read_csv("d:\\data\\emp.csv")
df = emp.loc[:, ['deptno', 'job', 'sal']]

grouped = df.groupby(['deptno', 'job'])

print(grouped.agg(sum)) # 정렬까지 아주 친절하게 해줌 (SQL> orderby까지 자동으로 잘해줌!)

                   sal
deptno job            
10     CLERK      1300
       MANAGER    2450
       PRESIDENT  5000
20     ANALYST    6000
       CLERK      1900
       MANAGER    2975
30     CLERK       950
       MANAGER    2850
       SALESMAN   5600


## 문제183. 아래의 SQL을 판다스로 변경하시오!

In [9]:
"""
SQL>
select deptno, min(sal), max(sal)
    from emp
    group by deptno;
"""
# Pandas

import pandas as pd
emp = pd.read_csv("d:\\data\\emp.csv")
df = emp.loc[:,['deptno', 'sal']]

grouped = df.groupby(['deptno'])

print(grouped.agg(['max','min']))

         sal      
         max   min
deptno            
10      5000  1300
20      3000   800
30      2850   950


## 문제184. 타이타닉 운임클래스별로 각각 운임의 최대값과 최소값을 아래와 같이 출력하시오!

In [16]:
import pandas as pd

tat = pd.read_csv("d:\\data\\train.csv")
print(tat.columns)

df = tat.loc[:,['Pclass','Fare']]
grouped = df.groupby(['Pclass'])


print(grouped.agg(['max','min']))

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
            Fare     
             max  min
Pclass               
1       512.3292  0.0
2        73.5000  0.0
3        69.5500  0.0


## ■ 그룹연산

   - df.groupby
   - 그룹객체.함수()
   - 그룹객체.agg
   - 그룹객체.filter(조건식함수)

## ■ 그룹객체.filter(조건식함수)

그룹객체 filter() 메소드를 적용할 때 조건식을 가진 함수를 전달하면 조건인 참인 그룹만을 남긴다.

__문법 :__ group객체.filter(조건식함수)

### 예제 : 이름과 월급과 직업과 부서번호를 출력하는데 자기 부서번호의 평균월급이 2000보다 작은 사원들만 출력하시오!

In [32]:
"""
SQL>
select ename, sal, job, deptno
    from emp
    where deptno in ( select deptno
                        from emp
                        group by deptno
                        having avg(sal) < 2000);
    
"""
# Pandas
import pandas as pd
emp = pd.read_csv("d:\\data\\emp.csv")
df = emp.loc[:, ['ename','sal','job','deptno']]

grouped = df.groupby(['deptno'])

# result = grouped.filter(함수)
result = grouped.filter(lambda x: x['sal'].mean() < 2000) # (lambda x: x.sal.mean() < 2000)
print(result)

    ename   sal       job  deptno
1   BLAKE  2850   MANAGER      30
4  MARTIN  1250  SALESMAN      30
5   ALLEN  1600  SALESMAN      30
6  TURNER  1500  SALESMAN      30
7   JAMES   950     CLERK      30
8    WARD  1250  SALESMAN      30


### 문제185. 타이타닉 데이터에서 운임 클래스별로 평균나이가 30살보다 작은 운임클래스의 승객들의 이름과 성별과 나이와 운임과 생존여부를 출력하시오!

In [40]:
import pandas as pd

tat = pd.read_csv("d:\\data\\train.csv")
print(tat.columns, '\n')

df = tat.loc[:,['Age', 'Fare', 'Pclass']]
grouped = df.groupby(['Pclass'])

print(grouped.agg(['mean']), '\n')

result = grouped.filter(lambda x: x['Age'].mean() < 30)
print(result)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object') 

              Age       Fare
             mean       mean
Pclass                      
1       38.233441  84.154687
2       29.877630  20.662183
3       25.140620  13.675550 

      Age     Fare  Pclass
0    22.0   7.2500       3
2    26.0   7.9250       3
4    35.0   8.0500       3
5     NaN   8.4583       3
7     2.0  21.0750       3
..    ...      ...     ...
884  25.0   7.0500       3
885  39.0  29.1250       3
886  27.0  13.0000       2
888   NaN  23.4500       3
890  32.0   7.7500       3

[675 rows x 3 columns]


## ■ 그룹객체.apply

apply() 메소드는 판다스 객체의 개별 원소를 특정 함수에 일대일로 매핑한다.

__문법 :__ group객체.apply(매핑함수)

### 예제 : class 열을 기준으로 구분한 3개의 그룹에 요약통계정보를 확인할 수 있다.

grouped = df.groupby(['Pclass'])  
result = grouped.apply(lambda x:x.describe())  
print(result)  

In [46]:
import pandas as pd

tat = pd.read_csv("d:\\data\\train.csv")
print(tat.columns, '\n')

df = tat.loc[:,['Age', 'Fare', 'Pclass']]
grouped = df.groupby(['Pclass'])

result = grouped.apply(lambda x:x.describe())  
print(result)  

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object') 

                     Age        Fare  Pclass
Pclass                                      
1      count  186.000000  216.000000   216.0
       mean    38.233441   84.154687     1.0
       std     14.802856   78.380373     0.0
       min      0.920000    0.000000     1.0
       25%     27.000000   30.923950     1.0
       50%     37.000000   60.287500     1.0
       75%     49.000000   93.500000     1.0
       max     80.000000  512.329200     1.0
2      count  173.000000  184.000000   184.0
       mean    29.877630   20.662183     2.0
       std     14.001077   13.417399     0.0
       min      0.670000    0.000000     2.0
       25%     23.000000   13.000000     2.0
       50%     29.000000   14.250000     2.0
       75%     36.000000   26.000000     2.0
       max     70.000000   73.500000     2.0
3      count  355.000000  491.00000

### apply 함수로 데이터 전처리시 유용하게 사용하는 코드는?

#### 데이터 정규화 할때 유용함.

### 예제 

In [50]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
print(titanic.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


In [54]:
import seaborn as sns

tat = sns.load_dataset('titanic')
print(tat.columns, '\n')

df = tat.loc[:,['age', 'fare', 'class', 'fare', 'survived','sex']]
grouped = df.groupby(['class'])

def z_score(x):
    return (x-x.mean()) / x.std()

agg_grouped = grouped['age'].apply(z_score)
print(agg_grouped.describe()) # 통계정보 확인 describe()

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object') 

count    7.140000e+02
mean    -7.945714e-17
std      9.985965e-01
min     -2.520692e+00
25%     -5.714600e-01
50%     -6.268304e-02
75%      5.922208e-01
max      3.910190e+00
Name: age, dtype: float64


## 문제186. 타이타닉 데이터의 운임 데이터를 정규화하시오!

In [60]:
import seaborn as sns

tat = sns.load_dataset('titanic')
print(tat.columns, '\n')

df = tat.loc[:,['age', 'fare', 'class', 'survived','sex']]
grouped = df.groupby(['class'])

def z_score(x):
    return (x-x.mean()) / x.std()

fare_grouped = grouped['fare'].apply(z_score)
print(fare_grouped.describe()) # 통계정보 확인 describe()

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object') 

count    8.910000e+02
mean     1.951301e-16
std      9.988758e-01
min     -1.539954e+00
25%     -5.455487e-01
50%     -4.475536e-01
75%      2.527764e-01
max      5.462777e+00
Name: fare, dtype: float64


## ■ 멀티 인덱스

groupby() 메소드에 여러 열을 리스트 형태로 전달하면 각 열들이 다중으로 행 인덱스를 구성하는 것을 보았다.  
이처럼 판다스는 행 인덱스를 여러 레벨로 구현 할 수 있도록 멀티 인덱스 클래스를 지원합니다.

In [59]:
# 예제
import seaborn as sns
tat = sns.load_dataset('titanic')
df = tat.loc[:,['age', 'fare', 'class', 'survived','sex']]

grouped = df.groupby(['class', 'sex'])
gdf = grouped.mean()
print(gdf)

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


## 문제187. 위의 결과에서 first 클래스만 따로 검색하시오!

In [3]:
import seaborn as sns
tat = sns.load_dataset('titanic')
df = tat.loc[:,['age', 'fare', 'class', 'survived','sex']]

grouped = df.groupby(['class', 'sex'])
gdf = grouped.mean()
print(gdf)
print('\n\n')
print(gdf.loc['First'])

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447



              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852


## 문제188. 위의 결과 first 클래스의 female 데이터만 검색하시오!

In [64]:
import seaborn as sns
tat = sns.load_dataset('titanic')
df = tat.loc[:,['age', 'fare', 'class', 'survived','sex']]

grouped = df.groupby(['class', 'sex'])
gdf = grouped.mean()
print(gdf.loc[('First', 'female')], '\n')

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64


## 문제189. 남자 승객에 class 별 평균나이와 평균 운임 데이터만 검색하시오!

In [4]:
import seaborn as sns

tat = sns.load_dataset('titanic')
df = tat.loc[:,['age', 'fare', 'class', 'survived','sex']]

grouped = df.groupby(['class', 'sex'])
gdf = grouped.mean()
print(gdf, end = '\n\n')
print(gdf.xs('male', level = 'sex'))
# 나이가 많을수록 상대적으로 비싼요금을 지불하고 높은 객실 등급을 이용하는 경향을 보이고 있다.

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

              age       fare  survived
class                                 
First   41.281386  67.226127  0.368852
Second  30.740707  19.741782  0.157407
Third   26.507589  12.661633  0.135447


### 문제190. 사원 테이블에서 직업별 평균 월급과 평균 커미션이 아래와 같이 출력되게 하시오.
### 어느 직업군이 월급이 가장 많고 커미션이 가장 많은지 확인.

In [78]:
import pandas as pd

emp = pd.read_csv("d:\\data\\emp.csv")
df = emp.loc[:,['sal','ename','comm','deptno','job']]

grouped = df.groupby(['job'])
gdf = grouped.mean().fillna(0)

print(gdf[['sal','comm']].sort_values(by = 'sal', ascending = False))

                   sal   comm
job                          
PRESIDENT  5000.000000    0.0
ANALYST    3000.000000    0.0
MANAGER    2758.333333    0.0
SALESMAN   1400.000000  550.0
CLERK      1037.500000    0.0


## ■ 8장. 머신러닝이란?

__스스로 데이터를 학습하여 서로 다른 변수간의 관계를 찾아 나가는 과정__

해결하려는 문제에 따라 __예측(prediction), 분류(classification), 군집(clustering)__ 등으로 분류된다.

__예를들면,__ (주가, 환율) 등 경제지표를 예측한다든가, 은행에서 고객을 분류하여 대출을 승인하거나 거절하는 문제, 비슷한 소비패턴을 가진 고객 유형을 군집으로 묶어내는 문제등이 있다.

머신러닝이 워낙 다양한 영역에 걸쳐있고, 사용하는 알고리즘과 방법론이 무수히 많기 때문에   
체계적으로 이론을 정립해 나가려면 상당한 시간과 노력이 필요하다.  

이 과정에서 많은 초심자들이 중간에 포기하거나 흥미를 잃게 된다.   
따라서 복잡한 이론보다는 실제 데이터를 가지고 간단한 문제부터 예측해보는 실습을 통해 익혀나가는것이 바람직하다.

## ■ 머신러닝의 종류 3가지 (OneNote 참고)

1. 지도학습
2. 비지도학습
3. 강화학습

## ■ 머신러닝 분석하는 과정

머신러닝 데이터 분석을 시작하기 전에 컴퓨터 알고리즘이 이해할수 있는 형태로 데이터를 변환하는 작업이 선행되어야 합니다.  
( 무엇으로 하는가? 판다스로 수행 )

#### < 데이터 전처리 >  
1. csv 파일을 데이터프레임으로 변환합니다.
2. 결측치를 제거하거나 치환합니다.
3. 범주형 데이터를 머신러닝 모델이 인식할 수 있도록 숫자형으로 변환합니다.
4. 정규화를 진행합니다.
5. 데이터를 훈련 데이터와 테스트 데이터로 나눕니다.

#### < 모델 생성 >  
6. 훈련 데이터로 모형을 학습하여 모형을 생성합니다.

#### < 모델 예측 및 평가 >  
7. 테스트 데이터를 훈련 모델을 넣어 예측을 합니다.
8. 모형의 예측능력을 평가합니다.

## ■ 9장. knn 알고리즘 이론 설명

__k nearest neighbor__의 약자로 k개의 최근 이웃이라는 뜻이다.  
머신러닝의 지도학습에 분류에 해당하는 알고리즘이다.  
새로 들어온 데이터가 기존 데이터의 그룹에 어느 그룹에 속하는지 찾을 때   
거리가 가까운 데이터의 그룹을 자기 그룹으로 선택하는 아주 간단한 알고리즘이다.

### ■ knn 알고리즘의 장단점

- 장점 : 단순하고 효율적이다. 훈련 단계가 빠르다.
- 단점 : 모델을 생성하지 않아서 특징과 클래스간의 관계를 이해하는 능력이 제약된다 & 적절한 k 값을 사용자가 직접 알아내야한다.

### ■ knn의 원리

새로 들어온 데이터가 기존의 데이터 중(악성종양, 양성종양) 에 어느 데이터에 더 인접해 있는지 거리를 계산해서   
가장 가까운 거리에 있는 데이터를 자기 이웃으로 선택한다.

__<거리를 계산할 때 사용하는 수학식?>__

__★유클리드 거리 계산식★__

### 예제1. 유클리드 거리 공식을 이용해서 토마토가 가장 가까운 거리에 있는 category가 무엇인지 파이썬으로 구현하시오!

In [80]:
import math
from collections import defaultdict
from collections import Counter

def knn(t,k):
    x = [8,2,7,7,3,1]
    y = [5,3,10,3,8,1]
    label = ['과일', '단백질', '채소', '과일', '채소', '단백질']    
    d = []
    
    box = defaultdict(list)
    
    for i in range(len(x)):
        x_t = (t[0] - x[i]) ** 2
        y_t = (t[1] - y[i]) ** 2
        result = round(math.sqrt(x_t + y_t),k)
        d.append(result) # 유클리드 기하학으로 나온거리를 리스트로 다 나타냄.
    
    s_d = sorted(d)[0:k] # k까지 순위 정해서 추출.
    
    for i, label in enumerate(label):
        box[label].append(d[i])
    
    li = []
    
    for i in box.keys():
        for j in range (len(box[i])):    
            if box[i][j] in s_d:
                li.append(i)
    print(li)
    
    li_s = Counter(li)
    
    print(li_s)
    
    tmp = list(li_s.keys())[0]
    
    return tmp
    

tomato = [6,4]

print ( knn(tomato,k=3) )

['과일', '과일', '단백질']
Counter({'과일': 2, '단백질': 1})
과일


## 예제2. 타이타닉 생존자를 예측하는 머신러닝 모델을 knn으로 구현하는 방법 총정리

In [9]:
import seaborn as sns
import pandas as pd

# 1단계. csv --> 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)
print(df.head(), '\n')

# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.

print(df.info()) # 자료형 확인하는 방법
print('\n')
# 2.2 결측치(NaN)을 확인한다.
print(df.isnull().sum(axis=0), '\n')

# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함
# embark 와 embark_town이 같은 데이터(중복데이터)여서 컬럼을 하나 삭제해야함.
# embark 컬럼은 삭제한다.

# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
rdf = df.drop(['deck', 'embark_town'], axis =1)
print(rdf.columns.values, '\n')

# 2.4 age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

rdf = rdf.dropna(subset = ['age'], how='any', axis = 0)
print(len(rdf), '\n')

# 설명
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace = True)
print(rdf.isnull().sum(axis=0), '\n')

# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
print(ndf.head(), '\n')
# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town') # 접두사를 정함.
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['sex', 'embarked'], axis = 1 , inplace = True)
print(ndf.head())
print('\n')

# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['survived'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
print(X)
print('\n')

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 10)

# 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
# random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.

print('train data 의 갯수:', X_train.shape) # (499,9)
print('test data 의 갯수:', X_test.shape) # (215, 9)
print('\n')

# 6단계. 머신러닝 모델을 생성한다.(knn을 사용)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5) # n_neighbors = 5 (knn의 k값은 5) 변수
knn.fit(X_train, y_train)

# 7단계. 테스트 데이터로 예측하기

y_hat = knn.predict(X_test)
print(y_hat[0:10])
print('\n')

# 8단계. 모형의 예측 능력을 평가한다.
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_hat,y_test)
print(knn_matrix)
print('\n')
#%%

# 8단계. 모형의 예측 능력을 평가한다.
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test,y_hat)
print(knn_matrix)
print('\n')

"""
             예측
           사망 생존
실제 사망 [[109  16]
     생존  [ 25  65]]
     
"""

# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
print(tn,fp,fn,tp)

f1_report = metrics.classification_report(y_test, y_hat)
print(f1_report,'\n')

# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)
print('\n')



# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

# precision(정밀도) 구하는 코드
from sklearn.metrics import precision_score
precision = precision_score(y_test,y_hat)
print(precision)
print('\n')

# recall(재현율) 구하는 코드
from sklearn.metrics import recall_score
recall = recall_score(y_test,y_hat)
print(recall)
print('\n')

# f1-score 구하는 코드
from sklearn.metrics import f1_score
f1 = f1_score(y_test,y_hat)
print(f1)

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age      

### ver2. 반복문과 최대값 함수를 추가하여 정확도 최대치를 뽑도록 Upgrade

In [8]:
import seaborn as sns
import pandas as pd

# 1단계. csv --> 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)
print(df.head(), '\n')

# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.

print(df.info()) # 자료형 확인하는 방법
print('\n')

# 2.2 결측치(NaN)을 확인한다.
print(df.isnull().sum(axis=0), '\n')

# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함
# embark 와 embark_town이 같은 데이터(중복데이터)여서 컬럼을 하나 삭제해야함.
# embark 컬럼은 삭제한다.

# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
rdf = df.drop(['deck', 'embark_town'], axis =1)


# 2.4 age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

rdf = rdf.dropna(subset = ['age'], how='any', axis = 0)


# 설명
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace = True)


# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked']]
print(ndf.head(), '\n')
# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town')
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['sex', 'embarked'], axis = 1 , inplace = True)


# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S']] # 독립변수
y = ndf['survived'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다. (반복문까지 추가함!!!!)

u = []
for i in range(10,30):
    
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    # 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
    # random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.

    
    # 6단계. 머신러닝 모델을 생성한다.(knn을 사용)
    for j in range(5,10):
        from sklearn.neighbors import KNeighborsClassifier
        
        knn = KNeighborsClassifier(n_neighbors= j) # n_neighbors = 5 (knn의 k값은 5) 변수
        knn.fit(X_train, y_train)
        
        # 7단계. 테스트 데이터로 예측하기
        
        y_hat = knn.predict(X_test)

   
        # 8단계. 모형의 예측 능력을 평가한다.
        from sklearn import metrics
        knn_matrix = metrics.confusion_matrix(y_test,y_hat)
        print('n_neighbors =', j, 'random_state =', i)
        print(knn_matrix)
        
        # 9단계 정확도 확인
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score( y_test, y_hat)
        print(accuracy)

        u.append('정확도는 ', accuracy)
        
        
        # precision(정밀도) 구하는 코드
        from sklearn.metrics import precision_score
        precision = precision_score(y_test,y_hat)

        print('precision은', precision)
 

        # recall(재현율) 구하는 코드
        from sklearn.metrics import recall_score
        recall = recall_score(y_test,y_hat)

        print('recall은', recall)


        # f1-score 구하는 코드
        from sklearn.metrics import f1_score
        f1 = f1_score(y_test,y_hat)

        print('f1-score 은' ,f1)

print('\n')
print('최대정확도는 : ', max(u))
        

# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age      

f1-score 은 0.7108433734939759
n_neighbors = 7 random_state = 18
[[99 17]
 [30 69]]
0.7813953488372093
precision은 0.8023255813953488
recall은 0.696969696969697
f1-score 은 0.745945945945946
n_neighbors = 8 random_state = 18
[[108   8]
 [ 36  63]]
0.7953488372093023
precision은 0.8873239436619719
recall은 0.6363636363636364
f1-score 은 0.7411764705882353
n_neighbors = 9 random_state = 18
[[100  16]
 [ 33  66]]
0.772093023255814
precision은 0.8048780487804879
recall은 0.6666666666666666
f1-score 은 0.7292817679558011
n_neighbors = 5 random_state = 19
[[117  17]
 [ 21  60]]
0.8232558139534883
precision은 0.7792207792207793
recall은 0.7407407407407407
f1-score 은 0.7594936708860759
n_neighbors = 6 random_state = 19
[[120  14]
 [ 27  54]]
0.8093023255813954
precision은 0.7941176470588235
recall은 0.6666666666666666
f1-score 은 0.7248322147651006
n_neighbors = 7 random_state = 19
[[118  16]
 [ 24  57]]
0.813953488372093
precision은 0.7808219178082192
recall은 0.7037037037037037
f1-score 은 0.7402597402597403


### 문제191. 유방암 데이터의 악성종양을 예측하는 머신러닝 모델을 생성하시오~
### 지금까지 배운 8단계를 그대로 적용해서 수행하세요~

__데이터 게시판 :__   
54번. 유방암 데이터

__데이터 설명 :__   
Wisconsin Hospital의 병원 데이터 유방암 종양의 크기와 지름, 거칠기 등에 대한 수치   
데이터 라벨 컬럼은 diagnosis이고 B가 양성 M이 악성임

#### 1단계 : 데이터 프레임 생성  
dataset = pd_read_csv("d:\\data\\wisc_bc_data.csv")

#### 2단계 : 결측치 확인 및 처리  
#### 3단계 : 범주형 데이터를 수치형 데이터로   
#### 4단계 : 정규화  
#### 5단계 : 훈련 데이터와 테스트 데이터를 나눈다.  (sklearn의 train_test_split)
X = dataset.iloc[:, 2:].values  
y = dataset.iloc[:, 1].values  

#### 6단계 : 훈련 데이터 모델 생성  (sklearn의 KNeighborsClassifier)
#### 7단계 : 테스트 데이터로 예측을 한다.  (sklearn의 predict)
#### 8단계 : 모형의 예측 능력을 평가한다.  (sklearn의 confustion_matrix)
#### 9단계 : 모델의 성능을 높인다.

   - 앙상블을 이용한다.
   - "파생변수"를 생성한다. (기존의 raw data로 새로운 학습 데이터를 생성해주어야한다.)

In [131]:
import seaborn as sns
import pandas as pd

# 1단계. csv --> 데이터 프레임으로 변환
df = pd.read_csv("d:\\data\\wisc_bc_data.csv")

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)
print(df.head(), '\n')


print(df.info())
print('\n') 


print(df.isnull().sum(axis=0), '\n')
print(df.columns)
print('\n')

# 2. 결측치를 제거하거나 치환 (결측치 값이 모든 열에 없기 때문에 과정 생략)
# 3. 범주형 데이터를 숫자형으로 변환하기 (문자로 나타나진 값이없어서 과정 생략)

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = df[['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se',
       'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'points_se', 'symmetry_se',
       'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst',
       'area_worst', 'smoothness_worst', 'compactness_worst',
       'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']]
print(ndf.head(), '\n')


# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'points_mean', 'symmetry_mean', 'dimension_mean', 'radius_se',
       'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'points_se', 'symmetry_se',
       'dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst',
       'area_worst', 'smoothness_worst', 'compactness_worst',
       'concavity_worst', 'points_worst', 'symmetry_worst', 'dimension_worst']] # 독립변수

y = ndf['diagnosis'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
print(X)
print('\n')

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 15)

# 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
# random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.

print('train data 의 갯수:', X_train.shape) # (499,9)
print('test data 의 갯수:', X_test.shape) # (215, 9)
print('\n')

# 6단계. 머신러닝 모델을 생성한다.(knn을 사용)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=28) # n_neighbors = 5 (knn의 k값은 5) 변수
knn.fit(X_train, y_train)

# 7단계. 테스트 데이터로 예측하기

y_hat = knn.predict(X_test)
print(y_hat[0:10])
print('\n')

# 8단계. 모형의 예측 능력을 평가한다.
from sklearn import metrics
knn_matrix = metrics.confusion_matrix(y_test,y_hat)
print(knn_matrix)
print('\n')

# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


         id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0  87139402         B        12.32         12.39           78.85      464.1   
1   8910251         B        10.60         18.95           69.28      346.4   
2    905520         B        11.04         16.83           70.92      373.2   
3    868871         B        11.28         13.39           73.00      384.8   
4   9012568         B        15.19         13.21           97.65      711.8   

   smoothness_mean  ...  area_worst  smoothness_worst  compactness_worst  \
0          0.10280  ...       549.1            0.1385             0.1266   
1          0.09688  ...       424.8            0.1213             0.2515   
2          0.10770  ...       471.4            0.1369             0.1482   
3          0.11640  ...       434.0            0.1367             0.1822   
4          0.07963  ...       819.1            0.1126             0.1737   

   concavity_worst  points_worst  symmetry_worst  dimension_worst  
